Trains a 2 head neural network classifier to predict whether a user will purchase a paired product

In [1]:
import pandas as pd, os, numpy as np
import plotly.express as px
pd.options.display.max_columns = 50
from tqdm.notebook import tqdm
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import regularizers
import pickle as pkl

In [2]:
df = pd.read_parquet('../data/sampleTrain.parquet')

In [3]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

In [4]:
train_indices = []
val_indices = []
for train_index, val_index in sss.split(df, df['Y']):
    train_indices.append(train_index)
    val_indices.append(val_index)

In [5]:
train_df = df.iloc[train_indices[0]].reset_index(drop=True).copy()
val_df = df.iloc[val_indices[0]].reset_index(drop=True).copy()

In [6]:
train_df.head()

,c_0,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,c_10,c_11,c_12,c_13,c_14,c_15,c_16,c_17,c_18,c_19,c_20,c_21,c_22,c_23,c_24,...,a_489,a_490,a_491,a_492,a_493,a_494,a_495,a_496,a_497,a_498,a_499,a_500,a_501,a_502,a_503,a_504,a_505,a_506,a_507,a_508,a_509,a_510,a_511,Y,Q
0,-0.007609,0.032056,-0.037316,-0.021332,0.047567,0.023968,0.053063,0.068663,0.020005,-0.021283,-0.055241,0.047815,0.034351,0.004660,0.054117,0.014661,0.060061,0.034148,0.027116,0.044250,-0.061070,0.037993,0.026378,-0.021512,0.035432,...,-0.045832,-0.010256,-0.021925,0.019728,-0.048264,-0.042763,0.022530,-0.050692,0.026909,0.032411,-0.054654,-0.048282,0.073165,0.014169,-0.046194,0.020750,0.049168,0.036420,-0.066694,0.012602,0.008862,-0.030414,0.027437,0,723819
1,-0.033950,0.057984,-0.071288,-0.043218,0.001485,0.071466,0.057743,0.082931,0.079448,-0.050903,-0.068545,0.055255,0.063914,-0.009430,0.052944,0.025674,0.038944,0.045122,0.012483,0.063228,-0.038934,0.054356,0.007674,-0.039047,0.012491,...,0.057799,0.017770,0.031712,0.033335,-0.053662,-0.017807,-0.039433,-0.040054,0.008012,0.009530,0.009211,-0.024321,0.071141,-0.035914,-0.022758,0.046278,0.047825,-0.070064,-0.068126,-0.001557,-0.016637,-0.051852,0.038415,0,934283
2,-0.042539,0.042159,0.020725,0.008295,-0.002174,-0.007319,0.039410,0.063293,0.029742,-0.001772,-0.046326,0.041718,0.005290,-0.000150,0.023447,0.025195,0.029470,0.043531,0.014864,0.040745,-0.061941,-0.010867,0.049788,-0.038780,0.027389,...,-0.019025,-0.003937,0.004133,-0.021396,-0.062676,0.006624,0.041481,0.021502,0.032379,-0.019148,-0.015441,-0.065368,0.076031,-0.053625,-0.063819,-0.027560,0.045164,-0.001526,-0.003567,-0.001584,-0.029391,-0.043211,0.024269,1,898888
3,-0.024661,0.033532,-0.037073,-0.029151,-0.004612,0.052987,0.077461,0.077839,0.082593,-0.058052,-0.034719,0.066648,0.061546,-0.001921,0.070346,0.062135,0.024700,0.030859,0.044692,0.054330,-0.053047,0.054074,0.019816,-0.030229,0.019991,...,0.006226,-0.027783,0.030283,0.020046,-0.052551,-0.037479,0.059561,-0.034385,0.000791,-0.033954,-0.038600,-0.036415,0.075786,0.019143,-0.059735,0.007111,0.069468,-0.072330,-0.064835,0.045485,-0.040182,-0.046414,0.029894,0,1340140
4,-0.020129,0.057922,-0.072309,-0.050506,0.046882,0.034397,0.069011,0.074125,0.066629,-0.059258,-0.038325,0.037379,0.070386,-0.042335,0.056509,0.045679,0.045406,0.017677,0.054679,0.063565,-0.029087,0.041543,0.038013,0.002830,-0.001023,...,0.037448,0.003398,0.036073,-0.025943,-0.057222,0.008719,0.003525,0.020458,0.020510,-0.017395,0.032116,0.018903,0.077522,-0.037799,-0.054228,-0.001145,0.065842,-0.072806,-0.036529,0.048998,-0.042488,-0.068159,0.050766,0,1059697


In [7]:
x_train = train_df.drop(['Y','Q'], axis=1).copy()
x_val = val_df.drop(['Y','Q'], axis=1).copy()

y_train = train_df['Y'].values
y_val = val_df['Y'].values

In [8]:
cf_xtrain = x_train[['c_' + str(x) for x in range(512)]].copy()
cf_xval = x_val[['c_' + str(x) for x in range(512)]].copy()

af_xtrain = x_train[['a_' + str(x) for x in range(512)]].copy()
af_xval = x_val[['a_' + str(x) for x in range(512)]].copy()

In [9]:
BATCH_SIZE = 64

In [10]:
fold_train_loss = []
fold_val_loss = []

train_dataset = tf.data.Dataset.from_tensor_slices(((cf_xtrain, af_xtrain), y_train))
val_dataset = tf.data.Dataset.from_tensor_slices(((cf_xval, af_xval), y_val))

train_dataset = train_dataset.batch(BATCH_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE)

train_dataset.prefetch(tf.data.AUTOTUNE)
val_dataset.prefetch(tf.data.AUTOTUNE)

<PrefetchDataset element_spec=((TensorSpec(shape=(None, 512), dtype=tf.float32, name=None), TensorSpec(shape=(None, 512), dtype=tf.float32, name=None)), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [21]:
def get_model():
    cf_inputs = tf.keras.Input((512, ))
    af_inputs = tf.keras.Input((512, ))
    
    cf_x = layers.Dense(25, activation='relu')(cf_inputs)
    cf_x = layers.Dense(12, activation='relu')(cf_x)
    
    af_x = layers.Dense(25, activation='relu')(af_inputs)
    af_x = layers.Dense(12, activation='relu')(af_x)
    
    fx = layers.Concatenate(axis=1)([cf_x, af_x])
    fx = layers.Dense(20, activation='relu', kernel_regularizer=regularizers.l2(0.001))(fx)
    fx = layers.Dense(10, activation='relu', kernel_regularizer=regularizers.l2(0.001))(fx)
    out = layers.Dense(1, activation='sigmoid', kernel_regularizer=regularizers.l2(0.001))(fx)
    
    model = tf.keras.Model(inputs=[cf_inputs, af_inputs], outputs=out)
    model.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.SGD(0.01),
                 metrics=['accuracy'])
    return model

In [22]:
keras.backend.clear_session()
dnn = get_model()
dnn.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 dense (Dense)                  (None, 25)           12825       ['input_1[0][0]']                
                                                                                                  
 dense_2 (Dense)                (None, 25)           12825       ['input_2[0][0]']                
                                                                                              

In [23]:
total = len(y_train)
pos = sum(y_train)
neg = total-pos

In [24]:
weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

Weight for class 0: 0.63
Weight for class 1: 2.45


In [25]:
1 - sum(y_train)/len(y_train)

0.7955937366883079

In [26]:
1 - sum(y_val)/len(y_val)

0.7955924180042564

In [27]:
history = dnn.fit(
    train_dataset,
    validation_data=val_dataset,
    verbose=1, epochs=30, workers=1, shuffle=False, class_weight=class_weight)


Epoch 1/30
9427/9427 [==============================] - 69s 7ms/step - loss: 0.7084 - accuracy: 0.5614 - val_loss: 0.6268 - val_accuracy: 0.6830
Epoch 2/30
9427/9427 [==============================] - 68s 7ms/step - loss: 0.6054 - accuracy: 0.6808 - val_loss: 0.5855 - val_accuracy: 0.6964
Epoch 3/30
9427/9427 [==============================] - 68s 7ms/step - loss: 0.5825 - accuracy: 0.7000 - val_loss: 0.5750 - val_accuracy: 0.7037
Epoch 4/30
9427/9427 [==============================] - 69s 7ms/step - loss: 0.5686 - accuracy: 0.7124 - val_loss: 0.5650 - val_accuracy: 0.7135
Epoch 5/30
9427/9427 [==============================] - 69s 7ms/step - loss: 0.5572 - accuracy: 0.7242 - val_loss: 0.5601 - val_accuracy: 0.7195
Epoch 6/30
9427/9427 [==============================] - 69s 7ms/step - loss: 0.5476 - accuracy: 0.7344 - val_loss: 0.5617 - val_accuracy: 0.7214
Epoch 7/30
9427/9427 [==============================] - 69s 7ms/step - loss: 0.5398 - accuracy: 0.7426 - val_loss: 0.5605 - val_ac

KeyboardInterrupt: 

In [28]:
dnn.save('../models/dnn1')

INFO:tensorflow:Assets written to: ../models/dnn1\assets


In [29]:
preds = dnn.predict(val_dataset, verbose=1)

2357/2357 [==============================] - 4s 2ms/step


In [31]:
preds = preds.flatten()

In [32]:
preds[0]

0.13393596

In [33]:
y_val[0]

0

In [39]:
preds[:40]

array([0.13393596, 0.26480663, 0.21671613, 0.14147896, 0.28341255,
       0.17072555, 0.14309306, 0.15220736, 0.5992796 , 0.22587974,
       0.30867004, 0.17417048, 0.06664606, 0.16928484, 0.93908167,
       0.7696869 , 0.8526217 , 0.3855799 , 0.0713409 , 0.92872137,
       0.95502585, 0.5422059 , 0.12449035, 0.12228905, 0.9467503 ,
       0.31891862, 0.13804312, 0.8031897 , 0.84503895, 0.40104243,
       0.5333662 , 0.20282413, 0.16526753, 0.8146628 , 0.18562071,
       0.71643573, 0.7633169 , 0.396171  , 0.63372844, 0.16468744],
      dtype=float32)